# Gen AI Model Deployment on Cloud

In [2]:
# Installing and Importing Libraries
!pip install -q --upgrade sagemaker

import sagemaker
from sagemaker.huggingface import HuggingFaceModel
import boto3 # The main AWS SDK

print(f"SageMaker version: {sagemaker.__version__}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
SageMaker version: 2.253.1


In [3]:
# Setting up the session
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"SageMaker Role ARN: {role}")
print(f"SageMaker Session Bucket: {sess.default_bucket()}")

SageMaker Role ARN: arn:aws:iam::352731040055:role/service-role/AmazonSageMaker-ExecutionRole-20251023T153336
SageMaker Session Bucket: sagemaker-us-east-2-352731040055


In [4]:
# Creating an instance of Hugging Face Model
hub = {
    'HF_MODEL_ID':'gpt2',
    'HF_TASK':'text-generation'
}

huggingface_model = HuggingFaceModel(
    env=hub,                        
    role=role,                      
    transformers_version="4.26",    
    pytorch_version="1.13",         
    py_version="py39"               
)

print("Model blueprint created successfully.")



Model blueprint created successfully.


In [5]:
# Deploying the Model
print("deploying the model's endpoint...")
predictor = huggingface_model.deploy(
    initial_instance_count=1,         # We just want one server
    instance_type="ml.m5.large"       # The hardware for our API
)
print(f"Endpoint '{predictor.endpoint_name}' is now InService!")

deploying the model's endpoint...
------!Endpoint 'huggingface-pytorch-inference-2025-10-23-20-12-43-039' is now InService!


In [ ]:
# Benchmarking the Sagemaker Endpoint
import time
print("--- Starting SageMaker Endpoint Benchmark ---")

data = {
   "inputs": "Explain quantum computing in one sentence"
}
num_requests = 10
latencies = []

print(f"Running benchmark with {num_requests} requests...")
for i in range(num_requests):
    start_time = time.time()
    result = predictor.predict(data=data)
    latency_ms = (time.time() - start_time) * 1000
    latencies.append(latency_ms)
    print(f"Request {i+1}/{num_requests} - Latency: {latency_ms:.2f} ms")
    
if latencies:
    avg_latency = sum(latencies) / len(latencies)
    total_time = sum(latencies) / 1000
    throughput = num_requests / total_time
    
    print("\n--- Benchmark Complete ---")
    print(f"Total Requests: {num_requests}")
    print(f"Average Latency: {avg_latency:.2f} ms")
    print(f"Total Time: {total_time:.2f} s")
    print(f"Throughput (Req/Sec): {throughput:.2f}")

--- Starting SageMaker Endpoint Benchmark ---
Running benchmark with 10 requests...
Request 1/10 - Latency: 2372.81 ms
Request 2/10 - Latency: 2217.55 ms
Request 3/10 - Latency: 2215.60 ms
Request 4/10 - Latency: 2209.65 ms
Request 5/10 - Latency: 2194.76 ms


In [ ]:
print(f"Deleting endpoint: {predictor.endpoint_name}...")
predictor.delete_endpoint()
print("Endpoint successfully deleted.")